This notebook makes use of the following modules: 

* urlib
* time
* os
* xml

These modules can be installed by running either one of the cells below. 

In [ ]:
import sys
!{sys.executable} -m pip install Twython

In [ ]:
import sys
!conda install --yes --prefix {sys.prefix} urllib
!conda install --yes --prefix {sys.prefix} time
!conda install --yes --prefix {sys.prefix} Twython

# A guessing game

The code below contains the code for a simple game in which the user needs to guess a number in between 1 and 50. 

The standard function input() can be used to request a value from the user. The function int() converts the input into an integer, if possible. The code also use of ‘for’ and of ‘while’. When the user enters a value which is too low or too high, this information is communicated to the user via a print statement. 

In [ ]:
number = 17

guess = int( input("Guess a number in between 1 and 50: ") )

while guess != number:

    if guess > number:
        print("Lower!")
    elif guess < number:
        print("Higher!")
    guess = int( input("Guess again!: ") )

print("The correct number is indeed {}.".format(number) )

# Data acquisition

The methode urlopen(), from the module urllib.request can be used to download files from the web. The code in the cell below downloads a given text from project Gutenberg.

In [ ]:
import urllib.request


url = "http://www.gutenberg.org/files/98/98-0.txt"

request = urllib.request.urlopen(url)
bytes = request.read()
contents = bytes.decode("utf-8")
request.close()

print(contents)


The code below cownloads a collection of texts. When you run the code, the texts are saved on your computer, in the same folder as this notebook.

In [ ]:
import urllib.request
import re
import time

def download( url ):

    request = urllib.request.urlopen(url)
    bytes = request.read()
    fullText = bytes.decode("utf-8")
    request.close()

    parts = re.split( '/' , url )
    id = parts[-1]

    out = open( id , 'w' , encoding = 'utf-8')
    out.write( fullText )
    out.close()
    time.sleep( 3 )



urls = [ 'http://www.gutenberg.org/files/580/580-0.txt' ,
'http://www.gutenberg.org/files/1400/1400-0.txt' ,
'http://www.gutenberg.org/files/786/786-0.txt' ,
'http://www.gutenberg.org/files/766/766-0.txt' 
]

for item in urls:
    print("downloading " + item + " ...")
    download(item)

# Wikipedia API

In [ ]:
import urllib.request
import json
import pprint

apiURL = 'https://en.wikipedia.org/w/api.php?action=opensearch&format=json&search='
searchTerm = "maastricht+leiden"

apiCall = apiURL + searchTerm

wikiHeader = {'User-Agent':'p.a.f.verhaar@hum.leidenuniv.nl'}

wikiRequest = urllib.request.Request(apiCall, headers=wikiHeader)

request = urllib.request.urlopen(wikiRequest)
responseData = request.read().decode()
request.close()


wikiResults = json.loads(responseData)


for i in range( 0 , len(wikiResults[1]) ):
    print( 'Title: ' + wikiResults[1][i] )
    print( 'Tagline: ' + wikiResults[2][i] )
    print( 'Url: ' + wikiResults[3][i] + '\n')


# Twitter API

In [ ]:
from twython import Twython, TwythonError
import re
import time

#fileName = time.strftime("%d%m%Y%H%M%S")


t = Twython(app_key= 'GA4x6zejqGBIs3wBJ44uRi9aq',
            app_secret='nU1oY1xhI9QGSYeqNFaD5Q7NU4SV0WCWNyyWwo7aqyrViDDz3d',
            oauth_token='17575499-PQsmouKqQ5E15lZq13dDHizX6LQ9dFb0hZAzTg501',
            oauth_token_secret='h4bi6r3HMkMBTQ9XEGHzsqyBzSMas8v6V3iQ0JzUbrtXX')

out = open("tweets.txt" , "w")


tag =  "leidenUniversity"


try:
    search = t.search(q='#' + tag  )
    tweets = search['statuses']
except TwythonError as e:
    print( e )


for tweet in tweets:
    out.write( tweet['id_str'] + "\t" )


    out.write( tweet['created_at'] + "\t" )
    out.write( tweet['user']['name'] + "\t" )
    out.write( tweet['user']['location'] + "\t" )
    tweet['text'] = re.sub( r'\s+' , ' ' , tweet['text']  )
    out.write( tweet['text'] + '\n' )


out.close()

# ORCID API

In [ ]:
orcid = '0000-0002-8469-6804'


import re
import urllib.request
import xml.etree.ElementTree as ET


ns = {'o': 'http://www.orcid.org/ns/orcid' ,
's' : 'http://www.orcid.org/ns/search' ,
'h': 'http://www.orcid.org/ns/history' ,
'p': 'http://www.orcid.org/ns/person' ,
'pd': 'http://www.orcid.org/ns/personal-details' ,
'a': 'http://www.orcid.org/ns/activities' ,
'e': 'http://www.orcid.org/ns/employment' ,
'c': 'http://www.orcid.org/ns/common' , 
'w': 'http://www.orcid.org/ns/work'}

def getXml( url):
    fp = urllib.request.urlopen( url )
    mybytes = fp.read()
    xml = mybytes.decode("utf8")
    fp.close()
    root = ET.fromstring(xml)
    return root

try:
    orcidUrl = "https://pub.orcid.org/v2.0/" + orcid
    
    xml = getXml( orcidUrl )
    
    creationDate = xml.find('h:history/h:submission-date' , ns ).text
    
    print('\nORCID created on:')
    print(creationDate)
    
    print('\nWorks:')
    
    works = xml.findall('a:activities-summary/a:works/a:group' , ns )
    for w in works:
        title = w.find('w:work-summary/w:title/c:title' , ns ).text
        print(title)
        doiEl = w.find('c:external-ids/c:external-id/c:external-id-url' , ns )
        if doiEl is not None:
            doi = doiEl.text
            print(doi)
            
except:
    print("Data could not be downloaded")


# OpenStreetMap API

In [ ]:

import urllib.request
import xml.etree.ElementTree as ET
import re
import string
from os.path import isfile, join , isdir
import os

addresses = ['Grote Looiersstraat 17 Maastricht' , 'Witte Singel 27 Leiden']


for a in addresses:
    url = 'https://nominatim.openstreetmap.org/search?q='+ a + '&format=xml'
    url = re.sub( '\s+' , '%20' , url )

    fp = urllib.request.urlopen( url )
    mybytes = fp.read()
    result = mybytes.decode("utf8")
    fp.close()
    print( result )
    root = ET.fromstring(result)
    el = root.findall('place')
    
    count = 0
    if el is not None:
        for place in el:
            count += 1
            lat = place.attrib['lat']
            lon = place.attrib['lon']
            if count == 1:
                print( '{},{}\n'.format( lat , lon ) )



# IIIF Image API

Documentation: https://iiif.io/api/image/2.1/

The following images (from the collection of the National Gallery of Art in Washingtn) are made available via IIIF.

https://media.nga.gov/iiif/public/objects/1/0/6/3/8/2/106382-primary-0-nativeres.ptif/full/full/0/default.jpg

https://media.nga.gov/iiif/public/objects/4/6/3/0/3/46303-primary-0-nativeres.ptif/full/full/0/default.jpg

https://media.nga.gov/iiif/public/objects/1/1/3/8/1138-primary-0-nativeres.ptif/full/full/0/default.jpg


Basic structure of URL according to IIIF Image API:
{scheme}://{server}{/prefix}/{identifier}/{region}/{size}/{rotation}/{quality}.{format}


You can request other manifestations of these images via the following parameters:

* Region: Value 'full' or four numbers. In the latter case, the first two numbers define the starting point and the last two number specify the width and the height. 
* Size: Value 'full or 'two numbers which specify the width and the height. The second number can be omitted
* Rotation: Number in between 0 or 359
* Format: values 'default','gray' or 'bitonal', followed by extensons 'jpg', 'png' or 'gif'

Sample query: 
https://media.nga.gov/iiif/public/objects/1/1/3/8/1138-primary-0-nativeres.ptif/20500,2000,20000,5000/500,/90/default.jpg

Example of Manuscript reconstruction:Châteauroux, Bibliothèque municipale, ms. 5 (Grandes Chroniques de France)
https://demos.biblissima.fr/chateauroux/demo/


## IIIF Exercises

Select one of the images listed above and try to request a version of the image with the following properties:
*	Bitonal, rotated 90 degrees
*	Image with a width of 150 pixels (default size for a thumbnail)
*	Zoom in on a specific detail of the image by specifying a region.

You can test the API query in the cell below.

![IIIF IMAGE](https://media.nga.gov/iiif/public/objects/1/1/3/8/1138-primary-0-nativeres.ptif/20500,2000,20000,5000/500,/0/grey.jpg)

# Webscraping

In [ ]:
from bs4 import BeautifulSoup
import urllib.request
import re


soup = ""

## Project Gutenberg Bookshelves can be found at 
## https://www.gutenberg.org/wiki/Category:Bookshelf

url = 'https://www.gutenberg.org/wiki/Philosophy_(Bookshelf)'



request = urllib.request.urlopen(url)
bytes = request.read()
contents = bytes.decode("utf-8")
request.close()


soup = BeautifulSoup(contents,"lxml")


links = soup.find_all("a")

for l in links:
    linktext = l.string
    url = l.get("href")

    if re.search('gutenberg' , str(url) , re.IGNORECASE):
        print(f"{linktext}: {url}")
